In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#default_exp search

In [3]:
#export
import sys
sys.path.insert(0,'..')
from mytools.tools import *
from food.paths import *
from food.psql import *
import requests
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from food.paths import *
from stego.segment import get_food_segment
from segmentor.segment import *

from mytools.visual import *

dev
Since no pretrained weights have been provided, we load the reference pretrained DINO weights.


/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (UnsupervisedMetrics). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_no_full_state`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


/home/dima/FoodSeg103-Benchmark-v1/checkpoints/SETR_Naive_ReLeM


/home/dima/FoodSeg103-Benchmark-v1/mmseg/models/builder.py:59: UserWarning: train_cfg and test_cfg is deprecated, please specify them in model
  warnings.warn(


load pre-trained weight from imagenet21k


In [4]:
#export
bad_cats = ['Vegetables on a sandwich','Candy containing chocolate','Baby juice']
bad_descs = ['Banana, fried']


foods = pd.read_sql("""select f.*
                        from food.foods_prompted f""",engine)

# foods = foods.drop(columns = ['clip'])
foods = foods.set_index('id')
foods = foods.dropna()

foods = foods[~foods['category'].isin(bad_cats)]
foods = foods[~foods['description'].isin(bad_descs)]

food_clips = series2tensor(foods['clip'])

/home/dima/mytools/mytools/tools.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])


In [39]:
#export
def search(url):
    img = get_image_from_url(url)
    img = crop_image_to_square(img)
    photo_id = url.split('/')[-1]
    i = np.asarray(img, dtype="uint8")
    i = np.flip(i,2)
    segmentor_mask = inference_segmentor(segment_model, i)[0]

    classes = np.unique(segmentor_mask)[1:]
    classes_ =[]
    urls = []
    for c in classes:
        area = segmentor_mask[segmentor_mask==c].shape[0]
        if area> 20*20:
            class_mask = np.where(segmentor_mask==c,1,0)
            class_mask = expand_boundaries(class_mask,times=5,factor=5)
            img_arr = apply_mask(img,class_mask.T).astype(np.uint8)
            img_arr = crop_zeros(img_arr)
            img_arr[img_arr==[0,0,0]]=255 #replace black with while
            fname = f'{photo_id}_{c}.jpg'

            Image.fromarray(img_arr).save(fragment_reference_images_path/fname)
            urls.append(f'https://dima.grankin.eu/fragment_reference_images/{fname}')
            classes_.append(c+1)

    classes = classes_
    stego_img,stego_mask = get_food_segment(img)
    stego_img.save(fragment_reference_images_path/f'{photo_id}_stego.jpg')
    urls.append(f'https://dima.grankin.eu/fragment_reference_images/{photo_id}_stego.jpg')

    clip_df = pd.DataFrame()
    for u in urls:
        clip_df = clip_df.append(search_clip(u,foods,food_clips,head = 1)[1])
    clip_df=clip_df.reset_index(drop=True)
    clip_df['classes'] = classes+[1]

    mask = stego_mask+segmentor_mask
    calorie_mask   = mask
    fat_mask       = mask
    carbs_mask     = mask
    protein_mask   = mask
    
    
    dicts =[]
    masks =[]
    
    for col in ['energy','protein','carb','fat']:
        dicts.append(clip_df[['classes',col]].set_index("classes")[col].to_dict())
        masks.append(torch.clone(mask))
        
    for c in np.unique(mask):
        if c not in dicts[0].keys():
            for mask in masks:
                mask[mask==c]=0
                
                
    for d,mask in zip(dicts,masks):
        for k,v in d.items(): mask[mask == k] = v
        
    
    return img,clip_df,masks

In [40]:
[f.unlink() for f in fragment_reference_images_path.ls()]

[None, None, None, None]

In [41]:
url = 'https://staticfanpage.akamaized.net/wp-content/uploads/sites/22/2022/03/Porridge-9-1200x675.jpg'

In [43]:
img,clip_df,masks = search(url)

/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/torch/nn/functional.py:3509: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/torch/nn/functional.py:3679: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
/home/dima/anaconda3/envs/f4/lib/python3.9/site-packages/torchvision/transforms/transforms.py:287: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/home/dima/anaconda3/envs/f

In [57]:
[round(float(mask[mask!=0].mean()),1) for mask in masks]

[64.1, 1.7, 10.7, 1.9]

In [48]:
visualize_array(masks[4].numpy(),img)

IndexError: list index out of range

In [9]:
' '.join(df['description'].tolist())

'Berries, NFS Berries, NFS Oatmeal, NS as to regular, quick, or instant, NS as to fat Oatmeal, NS as to regular, quick, or instant, NS as to fat'

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [12]:
mask[mask!=0].mean()

tensor(64.1507, dtype=torch.float64)

In [11]:
df

,id,description,category,energy,protein,carb,fat,text,clip,score,classes
0,1102699,"Berries, NFS",Blueberries and other berries,40.0,0.69,9.72,0.31,the food is blueberries and other berries . it...,"[0.045548365, -0.0103531685, 0.019558156, 0.00...",0.260989,32
1,1102699,"Berries, NFS",Blueberries and other berries,40.0,0.69,9.72,0.31,the food is blueberries and other berries . it...,"[0.045548365, -0.0103531685, 0.019558156, 0.00...",0.266057,33
2,1101576,"Oatmeal, NS as to regular, quick, or instant, ...",Oatmeal,76.0,2.18,11.17,2.61,the food is oatmeal . it has a little bit of ...,"[0.0011570271, 5.0268736e-06, -0.0030068187, 0...",0.280790,53
3,1101576,"Oatmeal, NS as to regular, quick, or instant, ...",Oatmeal,76.0,2.18,11.17,2.61,the food is oatmeal . it has a little bit of ...,"[0.0011570271, 5.0268736e-06, -0.0030068187, 0...",0.284375,1


In [42]:
!conda activate f4; nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_psql.ipynb.
Converted 01_search.ipynb.
Converted bot.ipynb.
Converted inference.ipynb.
Converted search_dev.ipynb.
Converted search_segmented_stable.ipynb.
Converted stats.ipynb.
Converted usda_parsing.ipynb.
